# Daily Stock Prices
##### Purpose: Pull daily stock data using LSEG API for specified rics 
##### Input: List of stock closing price data from start to end date 
##### For information on the LSEG Eikon API: 
#####         --General: https://developers.lseg.com/en/api-catalog/eikon/eikon-data-api
#####         --API Key: https://developers.lseg.com/en/article-catalog/article/consume-real-time-data-with-refinitiv-data-platform
##### Contact: brds@hbs.edu


## Set Up

### Import Libraries

In [1]:
## Import 
import eikon as ek # Eikon Python wrapper package
import pandas as pd 
from datetime import datetime, timedelta
import sys
import time
import logging
import IPython


### Set API Key

In [2]:
# Set API Key (see notes at top for how to obtain your API Key)
api_key_file = "Data\API_Key.txt"
with open(api_key_file, 'r') as file:
    api_key = file.read().strip()

ek.set_app_key(api_key)

# Or set API Key:  ek.set_app_key('YOUR_KEY_HERE') 

## Parameters

### Import Data

In [3]:
# Import Ric List 
ric_list = pd.read_excel("Data\Rics_List.xlsx")
display(ric_list.head(5))


,RIC
0,POOL.OQ
1,CHRW.OQ
2,AJG.N
3,CNP.N
4,AMCR.N


### Parameters

In [4]:
# Set Values
## Chunk Size (set how many RICs to pull at once, based off how many days being pulled -- this helps respect API limits)
chunk_size = 100 # Set Chunk size to respect API limits

## Currency 
currency = 'USD' 

## Dates
start_date_str = '2024-08-01'
end_date_str = '2024-08-15'

## Retry attempts if API fails 
max_retries = 3

# Create Date variables from strings 
start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
dates = pd.date_range(start=start_date_str, end=end_date_str).to_frame(index=False, name="Date")




## Pull down data

In [5]:
# Pull Down Vars
## Loop thru each date (one day at a time), for chunk_size number of rics at a time 

## Set Loop Vars
last = len(ric_list)
data = pd.DataFrame()


## Loop thru all dates 
current_date = start_date
while current_date <= end_date: 
    # Set Dates:
    current_date_str = current_date.strftime('%Y-%m-%d') # String of current date being looped on
    close_param = f'TR.PriceClose(SDate={current_date_str},EDate={current_date_str},Frq=D, CURN:{currency})' # Parameter into get_data (closing price)
    date_param = f'TR.PriceClose(SDate={current_date_str},EDate={current_date_str}, Frq=D).date' # Parameter entered into get_data (date of closing price)

    ## Loop thru all rics
    for start in range(0, last, chunk_size):
        retries=0
        end = start + chunk_size 
        ric_chunk = ric_list['RIC'].iloc[start:end].tolist() #The rics being pulled in this iteration of the loop 

        #Try pulling data 3 times (sometimes API has too many requests)
        while retries < max_retries:
            try: 
                ## Pull Data 
                print(f"Pulling...")
                data_chunk,e = ek.get_data(ric_chunk,
                            [date_param,
                             close_param])
                
                break


            except Exception as e: 
                retries+=1 
                print(f"Attempt {retries} failed with error: {e}")
                if retries < max_retries: 
                    if retries < 3: 
                        wait_time = 15*retries 
                    else: 
                        wait_time = 60

                    print(f"Waiting {wait_time} seconds before retrying pull...")
                    time.sleep(wait_time) 

                else:
                    print("Max retries reached. Moving to next chunk of rics")                 

        ## Create date column for the date we are requesting data for (
        data_chunk['Date Requested'] = current_date

        data = pd.concat([data, data_chunk])

        print("Pulled - For date: ", current_date, "Rows: ", end)

        
#                print("Error with date:", current_date, "  For rows: ", start, " to ", end)
        
        # Adhere to API limits 
        time.sleep(2) 

    # Increment by 1 day and record data obtained 
    print("Obtained data for: ", current_date)
    current_date += timedelta(days=1)

print ("Finished pulling data")

Pulling...
Pulled - For date:  2024-08-01 00:00:00 Rows:  100
Pulling...
Pulled - For date:  2024-08-01 00:00:00 Rows:  200
Pulling...
Pulled - For date:  2024-08-01 00:00:00 Rows:  300
Pulling...
Pulled - For date:  2024-08-01 00:00:00 Rows:  400
Pulling...
Pulled - For date:  2024-08-01 00:00:00 Rows:  500
Pulling...
Pulled - For date:  2024-08-01 00:00:00 Rows:  600
Obtained data for:  2024-08-01 00:00:00
Pulling...
Pulled - For date:  2024-08-02 00:00:00 Rows:  100
Pulling...
Pulled - For date:  2024-08-02 00:00:00 Rows:  200
Pulling...
Pulled - For date:  2024-08-02 00:00:00 Rows:  300
Pulling...
Pulled - For date:  2024-08-02 00:00:00 Rows:  400
Pulling...
Pulled - For date:  2024-08-02 00:00:00 Rows:  500
Pulling...
Pulled - For date:  2024-08-02 00:00:00 Rows:  600
Obtained data for:  2024-08-02 00:00:00
Pulling...
Pulled - For date:  2024-08-03 00:00:00 Rows:  100
Pulling...
Pulled - For date:  2024-08-03 00:00:00 Rows:  200
Pulling...
Pulled - For date:  2024-08-03 00:00:00 R

2024-09-05 13:42:52,424 P[30220] [MainThread 72356] Error code 403 | Client Error: {"ErrorCode":1401,"ErrorMessage":"Application id 131a9d6912174284963b089ab481fc74d3348f7c is invalid"}
2024-09-05 13:42:52,426 P[30220] [MainThread 72356] HTTP request failed: EikonError-Client Error: {"ErrorCode":1401,"ErrorMessage":"Application id 131a9d6912174284963b089ab481fc74d3348f7c is invalid"}


Pulling...
Attempt 1 failed with error: Error code 403 | Client Error: {"ErrorCode":1401,"ErrorMessage":"Application id 131a9d6912174284963b089ab481fc74d3348f7c is invalid"}
Waiting 15 seconds before retrying pull...
Pulling...
Pulled - For date:  2024-08-09 00:00:00 Rows:  600
Obtained data for:  2024-08-09 00:00:00
Pulling...
Pulled - For date:  2024-08-10 00:00:00 Rows:  100
Pulling...
Pulled - For date:  2024-08-10 00:00:00 Rows:  200
Pulling...
Pulled - For date:  2024-08-10 00:00:00 Rows:  300
Pulling...
Pulled - For date:  2024-08-10 00:00:00 Rows:  400
Pulling...
Pulled - For date:  2024-08-10 00:00:00 Rows:  500
Pulling...
Pulled - For date:  2024-08-10 00:00:00 Rows:  600
Obtained data for:  2024-08-10 00:00:00
Pulling...
Pulled - For date:  2024-08-11 00:00:00 Rows:  100
Pulling...
Pulled - For date:  2024-08-11 00:00:00 Rows:  200
Pulling...
Pulled - For date:  2024-08-11 00:00:00 Rows:  300
Pulling...
Pulled - For date:  2024-08-11 00:00:00 Rows:  400
Pulling...
Pulled - F

2024-09-05 13:47:02,600 P[30220] [MainThread 72356] Error code 403 | Client Error: {"ErrorCode":1401,"ErrorMessage":"Application id 131a9d6912174284963b089ab481fc74d3348f7c is invalid"}
2024-09-05 13:47:02,602 P[30220] [MainThread 72356] HTTP request failed: EikonError-Client Error: {"ErrorCode":1401,"ErrorMessage":"Application id 131a9d6912174284963b089ab481fc74d3348f7c is invalid"}


Pulling...
Attempt 1 failed with error: Error code 403 | Client Error: {"ErrorCode":1401,"ErrorMessage":"Application id 131a9d6912174284963b089ab481fc74d3348f7c is invalid"}
Waiting 15 seconds before retrying pull...
Pulling...
Pulled - For date:  2024-08-14 00:00:00 Rows:  600
Obtained data for:  2024-08-14 00:00:00
Pulling...
Pulled - For date:  2024-08-15 00:00:00 Rows:  100
Pulling...
Pulled - For date:  2024-08-15 00:00:00 Rows:  200
Pulling...
Pulled - For date:  2024-08-15 00:00:00 Rows:  300
Pulling...
Pulled - For date:  2024-08-15 00:00:00 Rows:  400
Pulling...
Pulled - For date:  2024-08-15 00:00:00 Rows:  500
Pulling...
Pulled - For date:  2024-08-15 00:00:00 Rows:  600
Obtained data for:  2024-08-15 00:00:00
Finished pulling data


## Clean Data

### Rename/Reorder columns

In [11]:
# Clean up data
data_orig = data
data.rename(columns={'Date':'Date Obtained'}, inplace =True)
col_order = ['Instrument', 'Date Requested', 'Date Obtained', 'Price Close']
data = data[col_order]
data = data.sort_values(by=['Instrument', 'Date Requested'], ascending=[True, True])
data['Date Obtained'] = pd.to_datetime(data['Date Obtained']).dt.date
#data = data.drop_duplicates()       


#Compare original vs clean data 
data_orig.info()
data.info

<class 'pandas.core.frame.DataFrame'>
Index: 7545 entries, 97 to 3
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Instrument      7545 non-null   string        
 1   Date Requested  7545 non-null   datetime64[ns]
 2   Date Obtained   7545 non-null   object        
 3   Price Close     7545 non-null   Float64       
dtypes: Float64(1), datetime64[ns](1), object(1), string(1)
memory usage: 302.1+ KB


<bound method DataFrame.info of    Instrument Date Requested Date Obtained  Price Close
97        A.N     2024-08-01    2024-08-01       142.86
97        A.N     2024-08-02    2024-08-02       139.04
97        A.N     2024-08-03    2024-08-02       139.04
97        A.N     2024-08-04    2024-08-02       139.04
97        A.N     2024-08-05    2024-08-05        134.7
..        ...            ...           ...          ...
3       ZTS.N     2024-08-11    2024-08-09       185.25
3       ZTS.N     2024-08-12    2024-08-12       183.81
3       ZTS.N     2024-08-13    2024-08-13       188.38
3       ZTS.N     2024-08-14    2024-08-14       183.38
3       ZTS.N     2024-08-15    2024-08-15       184.08

[7545 rows x 4 columns]>

In [7]:
# Identify duplicates based on the combination of the pivoting indices and columns
duplicate_mask = data.duplicated(subset=['Date Requested', 'Instrument'], keep=False)

# Filter the DataFrame to get only the duplicated rows
duplicates = data[duplicate_mask]

# Print the duplicated rows
print(duplicates)

Empty DataFrame
Columns: [Instrument, Date Requested, Date Obtained, Price Close]
Index: []


### Reshape Data to Wide

In [12]:
## Reshape Data 
data_wide = data.pivot(index='Date Requested', columns = 'Instrument')


# Flatten the MultiIndex columns
data_wide.columns = [f'{instr}_{col}' for col, instr in data_wide.columns]


# Sort the columns based on instrument name
sorted_columns = sorted(data_wide.columns)
data_wide = data_wide[sorted_columns]

# Reset the index to turn 'Date Requested' back into a column
data_wide.reset_index(inplace=True)


### Reshape Data to Wide

## View and Export Data

In [13]:
# View some Data
display(data_wide.head(5))


,Date Requested,A.N_Date Obtained,A.N_Price Close,AAL.OQ_Date Obtained,AAL.OQ_Price Close,AAPL.OQ_Date Obtained,AAPL.OQ_Price Close,ABBV.N_Date Obtained,ABBV.N_Price Close,ABNB.OQ_Date Obtained,...,XYL.N_Date Obtained,XYL.N_Price Close,YUM.N_Date Obtained,YUM.N_Price Close,ZBH.N_Date Obtained,ZBH.N_Price Close,ZBRA.OQ_Date Obtained,ZBRA.OQ_Price Close,ZTS.N_Date Obtained,ZTS.N_Price Close
0,2024-08-01,2024-08-01,142.86,2024-08-01,10.14,2024-08-01,218.36,2024-08-01,189.71,2024-08-01,...,2024-08-01,133.33,2024-08-01,134.5,2024-08-01,111.12,2024-08-01,342.58,2024-08-01,182.42
1,2024-08-02,2024-08-02,139.04,2024-08-02,9.63,2024-08-02,219.86,2024-08-02,189.29,2024-08-02,...,2024-08-02,130.15,2024-08-02,135.18,2024-08-02,110.36,2024-08-02,322.3,2024-08-02,180.59
2,2024-08-03,2024-08-02,139.04,2024-08-02,9.63,2024-08-02,219.86,2024-08-02,189.29,2024-08-02,...,2024-08-02,130.15,2024-08-02,135.18,2024-08-02,110.36,2024-08-02,322.3,2024-08-02,180.59
3,2024-08-04,2024-08-02,139.04,2024-08-02,9.63,2024-08-02,219.86,2024-08-02,189.29,2024-08-02,...,2024-08-02,130.15,2024-08-02,135.18,2024-08-02,110.36,2024-08-02,322.3,2024-08-02,180.59
4,2024-08-05,2024-08-05,134.7,2024-08-05,9.28,2024-08-05,209.27,2024-08-05,184.36,2024-08-05,...,2024-08-05,126.14,2024-08-05,133.32,2024-08-05,107.04,2024-08-05,317.86,2024-08-05,174.82


In [15]:
data.to_excel('Data/Output_DailyStockPrices_Long.xlsx')
data_wide.to_excel('Data/Output_DailyStockPrices.xlsx')
